In [13]:
import yfinance as yf
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

def fetch_liquidity_ratios(ticker, year):
    try:
        stock = yf.Ticker(ticker)
        balance_sheet = stock.balance_sheet

        if balance_sheet.empty:
            print(f"No balance sheet data found for {ticker}. Try another ticker.")
            return

        balance_sheet = balance_sheet.T

        balance_sheet.index = pd.to_datetime(balance_sheet.index)

        selected_year = balance_sheet[balance_sheet.index.year == int(year)]

        if selected_year.empty:
            print(f"No balance sheet data found for {ticker} in {year}.")
            return

        total_current_assets = selected_year.get("Current Assets", pd.Series(0, index=selected_year.index))
        total_current_liabilities = selected_year.get("Current Liabilities", pd.Series(0, index=selected_year.index))
        inventory = selected_year.get("Inventory", pd.Series(0, index=selected_year.index))
        cash_equivalents = selected_year.get("Cash Cash Equivalents And Short Term Investments", pd.Series(0, index=selected_year.index))

        total_current_assets = total_current_assets.infer_objects(copy=False).fillna(0)
        total_current_liabilities = total_current_liabilities.infer_objects(copy=False).fillna(0)
        inventory = inventory.infer_objects(copy=False).fillna(0)
        cash_equivalents = cash_equivalents.infer_objects(copy=False).fillna(0)

        total_current_liabilities.replace(0, float("nan"), inplace=True)

        selected_year = selected_year.copy()
        selected_year.loc[:, "Current Ratio"] = (total_current_assets / total_current_liabilities).round(2)
        selected_year.loc[:, "Quick Ratio"] = ((total_current_assets - inventory) / total_current_liabilities).round(2)
        selected_year.loc[:, "Cash Ratio"] = (cash_equivalents / total_current_liabilities).round(2)

        print(f"Liquidity Ratios for {ticker} in {year}:")
        display(selected_year[["Current Ratio", "Quick Ratio", "Cash Ratio"]])

    except Exception as e:
        print("Error fetching data:", e)

ticker_input = widgets.Text(description="Ticker:")
year_input = widgets.Text(description="Year (YYYY):")
compute_button = widgets.Button(description="Compute")
output = widgets.Output()

def on_compute_button_clicked(b):
    with output:
        clear_output(wait=True)
        ticker = ticker_input.value.strip().upper()
        year = year_input.value.strip()

        if not ticker or not year.isdigit():
            print("Please enter a valid ticker and year.")
            return

        fetch_liquidity_ratios(ticker, year)

compute_button.on_click(on_compute_button_clicked)

display(ticker_input, year_input, compute_button, output)

Text(value='', description='Ticker:')

Text(value='', description='Year (YYYY):')

Button(description='Compute', style=ButtonStyle())

Output()